# Build Various Architectures

In [3]:
import numpy as np
import pickle
import keras
from keras import models, optimizers, layers, regularizers
from keras.models import model_from_json
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)
import matplotlib.pyplot as plt

IMG_SIZE = 256

In [2]:
# Save baseline model architecture and weights
def save_base(model_name):
    model_str = str(input("Save model as: "))

    # serialize model to JSON
    model_name = baseline.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    baseline.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Save Comparison model
def save_model(model_name):
    model_str = str(input("Save model as: "))

    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    
# Load model architecture and weights NOTE: must compile again
def load_model():
    model_str = str(input("Name of model to load: "))

    # load json and create model
    json_file = open('CNN Models/' + model_str + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("CNN Models/" + model_str + ".h5")
    print("Loaded " + model_str + " and weights from CNN Models folder")
    
    return loaded_model

# Write history object to a file using pickle
def save_history(model_name):
    hist_str = str(input("Save history as: "))

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
# Load history object
def load_history():
    hist_str = str(input("Name of history to load: "))

    loaded_history = pickle.load(open('Training Histories/' + hist_str + '.p', 'rb'))
    
    print("Loaded " + hist_str + " from Training Histories folder")
    
    return loaded_history

### Residual Network

In [3]:
'''
img_height = 256
img_width = 256
img_channels = 1

#
# network params
#

cardinality = 4


def residual_network(x):
    """
    ResNeXt by default. For ResNet set `cardinality` = 1 above.
    
    """
    def add_common_layers(y):
        #y = layers.BatchNormalization()(y)
        y = layers.ReLU()(y)

        return y

    def grouped_convolution(y, nb_channels, _strides):
        # when `cardinality` == 1 this is just a standard convolution
        if cardinality == 1:
            return layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
        
        assert not nb_channels % cardinality
        _d = nb_channels // cardinality

        # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
        # and convolutions are separately performed within each group
        groups = []
        for j in range(cardinality):
            group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
            groups.append(layers.Conv2D(_d, kernel_size=(3, 3), strides=_strides, padding='same')(group))
            
        # the grouped convolutional layer concatenates them as the outputs of the layer
        y = layers.concatenate(groups)

        return y

    def residual_block(y, nb_channels_in, nb_channels_out, _strides=(1, 1), _project_shortcut=False):
        """
        Our network consists of a stack of residual blocks. These blocks have the same topology,
        and are subject to two simple rules:
        - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
        - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
        """
        shortcut = y

        # we modify the residual building block as a bottleneck design to make the network more economical
        y = layers.Conv2D(nb_channels_in, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
        y = add_common_layers(y)

        # ResNeXt (identical to ResNet when `cardinality` == 1)
        y = grouped_convolution(y, nb_channels_in, _strides=_strides)
        y = add_common_layers(y)

        y = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=(1, 1), padding='same')(y)
        # batch normalization is employed after aggregating the transformations and before adding to the shortcut
        #y = layers.BatchNormalization()(y)

        # identity shortcuts used directly when the input and output are of the same dimensions
        if _project_shortcut or _strides != (1, 1):
            # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
            # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
            shortcut = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=_strides, padding='same')(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)

        y = layers.add([shortcut, y])

        # relu is performed right after each batch normalization,
        # expect for the output of the block where relu is performed after the adding to the shortcut
        y = layers.ReLU()(y)

        return y

    # conv block 1
    x = layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = add_common_layers(x)    
    x = layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='same')(x)
    
    # residual block
    for i in range(3):
        project_shortcut = (i == 0)
        x = residual_block(x, 32, 64, _project_shortcut=project_shortcut)

    # conv block 2
    x = layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = add_common_layers(x)    
    x = layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='same')(x)
    
    # conv block 3
    x = layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), padding='same')(x)
    x = add_common_layers(x)    
    x = layers.MaxPool2D(pool_size=(2, 2), strides=None, padding='same')(x)
    
    
    
    for i in range(4):
        # down-sampling is performed by conv3_1, conv4_1, and conv5_1 with a stride of 2
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 64, 128, _strides=strides)

    # conv4
    for i in range(6):
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 128, 256, _strides=strides)

    # conv5
    for i in range(3):
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 256, 512, _strides=strides)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(15)(x)

    return x


image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
network_output = residual_network(image_tensor)
  
model = models.Model(inputs=[image_tensor], outputs=[network_output])
print(model.summary())
'''

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 256, 256, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [4]:
#save_model(model)

Save model as: 3_vanilla_1_strawberry
Saved 3_vanilla_1_strawberry and weights to CNN Models folder


### Vanilla ConvNets

In [4]:
# Smallest Capacity
tiny_vanilla = models.Sequential()
tiny_vanilla.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape= (IMG_SIZE, IMG_SIZE, 1)))
tiny_vanilla.add(layers.Conv2D(16, (3, 3), activation='relu'))
tiny_vanilla.add(layers.MaxPooling2D(pool_size=(2, 2)))

tiny_vanilla.add(layers.Flatten())
tiny_vanilla.add(layers.Dense(15, activation='sigmoid'))

tiny_vanilla.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

tiny_vanilla_history = tiny_vanilla.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# tiny_vanilla

Save model as: tiny_vanilla
Saved tiny_vanilla and weights to CNN Models folder


In [ ]:
# 2 Layers, Small Capacity
double_tiny_vanilla = models.Sequential()
double_tiny_vanilla.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape= (IMG_SIZE, IMG_SIZE, 1)))
double_tiny_vanilla.add(layers.MaxPooling2D(pool_size=(2, 2)))
double_tiny_vanilla.add(layers.Conv2D(32, (3, 3), activation='relu'))
double_tiny_vanilla.add(layers.MaxPooling2D(pool_size=(2, 2)))

double_tiny_vanilla.add(layers.Flatten())
double_tiny_vanilla.add(layers.Dense(15, activation='sigmoid'))

double_tiny_vanilla.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

double_tiny_vanilla_history = double_tiny_vanilla.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# double_tiny_vanilla

In [ ]:
vanilla_reg_VGG = models.Sequential()
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))
#vanilla_reg_VGG.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))
#vanilla_reg_VGG.add(layers.Dropout(0.5))

vanilla_reg_VGG.add(layers.Flatten())
vanilla_reg_VGG.add(layers.Dense(64, activation='relu'))
vanilla_reg_VGG.add(layers.Dense(15, activation='sigmoid'))

vanilla_reg_VGG.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

vanilla_reg_VGG_history = vanilla_reg_VGG.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_reg_VGG

In [ ]:
vanilla_VGG_Dropouts = models.Sequential()
vanilla_VGG_Dropouts.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_VGG_Dropouts.add(layers.Conv2D(16, (3, 3), activation='relu'))
vanilla_VGG_Dropouts.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts.add(layers.Conv2D(32, (3, 3), activation='relu'))
vanilla_VGG_Dropouts.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))
vanilla_VGG_Dropouts.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts.add(layers.Conv2D(64, (3, 3), activation='relu'))
vanilla_VGG_Dropouts.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts.add(layers.Conv2D(128, (3, 3), activation='relu'))
vanilla_VGG_Dropouts.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_VGG_Dropouts.add(layers.Conv2D(256, (3, 3), activation='relu'))
vanilla_VGG_Dropouts.add(layers.MaxPooling2D((2, 2)))
vanilla_VGG_Dropouts.add(layers.Dropout(0.5))

vanilla_VGG_Dropouts.add(layers.Flatten())
vanilla_VGG_Dropouts.add(layers.Dense(64, activation='relu'))
vanilla_VGG_Dropouts.add(layers.Dense(15, activation='sigmoid'))

vanilla_VGG_Dropouts.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

vanilla_VGG_Dropouts_history = vanilla_VGG_Dropouts.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# vanilla_VGG_Dropouts

In [ ]:
VGG_Doppleganger = models.Sequential()
VGG_Doppleganger.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
VGG_Doppleganger.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))
VGG_Doppleganger.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))
VGG_Doppleganger.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))
VGG_Doppleganger.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))
VGG_Doppleganger.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))
VGG_Doppleganger.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
VGG_Doppleganger.add(layers.MaxPooling2D((2, 2)))
VGG_Doppleganger.add(layers.Dropout(0.5))

VGG_Doppleganger.add(layers.Flatten())
VGG_Doppleganger.add(layers.Dense(64, activation='relu'))
VGG_Doppleganger.add(layers.Dense(15, activation='sigmoid'))

VGG_Doppleganger.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

VGG_Doppleganger_history = VGG_Doppleganger.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

# VGG_Doppleganger

In [ ]:
# Save all histories
#save_history(tiny_vanilla)
#save_history(double_tiny_vanilla)
#save_history(vanilla_reg_VGG)
#save_history(vanilla_VGG_Dropouts)
#save_history(VGG_Doppleganger)